In [276]:
import socket
import pandas as pd
import numpy as np

In [277]:
key = [1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0]
chunk_size = 7

In [ ]:
def mapping(csv_filename, chunk_size=7):
    '''

    '''
    decoding_map = {}
    df =pd.read_csv(csv_filename)

    for index, row in df.iterrows():
        df['number'] =  pd.to_numeric(df['number']).astype(int)
        df['number'] = df['number'].apply(lambda x: bin(x & 0x7F)[2:].zfill(7))                                 
        decoding_map[row[0]] = row[1]

    print(decoding_map) 
    return decoding_map

In [279]:
HOST = '127.0.0.1'  # Dirección local (localhost)
PORT = 65432        # Puerto para la comunicación

In [280]:
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen()
    print("Bob esperando conexión...")
    conn, addr = s.accept()
    with conn:
        print(f"Conectado desde {addr}")
        encrypted_message = conn.recv(1024).decode()
        print("Mensaje cifrado recibido:", encrypted_message)

Bob esperando conexión...
Conectado desde ('127.0.0.1', 32904)
Mensaje cifrado recibido: 1110111011010010111000000100100011100100111111010100100010010101000110010000101110110100100001010101000011001001101111100011001100001101011110000010100111110111101100111001001


In [281]:
encrypted_bits = [int(digit) for digit in encrypted_message]

In [282]:
key_length = len(key)
extended_key = [key[i % key_length] for i in range(len(encrypted_bits))]

In [283]:
key_message = np.zeros(len(encrypted_bits), dtype=int)
for i in range(len(encrypted_bits)):
    if extended_key[i] == 1:
        key_message[i] = encrypted_bits[i] ^ 1
    else:
        key_message[i] = encrypted_bits[i]


In [284]:
flat_decrypted_string = "".join(map(str, key_message))

#print(f"Cadena binaria plana (total de bits: {len(flat_binary_string)}):\n{flat_binary_string[:70]}...") 

list_of_binary_strings_7bit = []
for i in range(0, len(flat_decrypted_string), chunk_size):
    chunk = flat_decrypted_string[i : i + chunk_size]
    list_of_binary_strings_7bit.append(chunk)

In [285]:
decoding_map_from_csv = mapping("alpha.csv", chunk_size)

     number character
0   0000001         a
1   0000010         b
2   0000011         c
3   0000100         d
4   0000101         e
5   0000110         f
6   0000111         g
7   0001000         h
8   0001001         i
9   0001010         j
10  0001011         k
11  0001100         l
12  0001101         m
13  0001110         n
14  0001111         o
15  0010000         p
16  0010001         q
17  0010010         r
18  0010011         s
19  0010100         t
20  0010101         u
21  0010110         v
22  0010111         w
23  0011000         x
24  0011001         y
25  0011010         z
26  0011011         A
27  0011100         B
28  0011101         C
29  0011110         D
30  0011111         E
31  0100000         F
32  0100001         G
33  0100010         H
34  0100011         I
35  0100100         J
36  0100101         K
37  0100110         L
38  0100111         M
39  0101000         N
40  0101001         O
41  0101010         P
42  0101011         Q
43  0101100         R
44  010110

In [286]:
decoding_map_from_csv

{1: 'a',
 2: 'b',
 3: 'c',
 4: 'd',
 5: 'e',
 6: 'f',
 7: 'g',
 8: 'h',
 9: 'i',
 10: 'j',
 11: 'k',
 12: 'l',
 13: 'm',
 14: 'n',
 15: 'o',
 16: 'p',
 17: 'q',
 18: 'r',
 19: 's',
 20: 't',
 21: 'u',
 22: 'v',
 23: 'w',
 24: 'x',
 25: 'y',
 26: 'z',
 27: 'A',
 28: 'B',
 29: 'C',
 30: 'D',
 31: 'E',
 32: 'F',
 33: 'G',
 34: 'H',
 35: 'I',
 36: 'J',
 37: 'K',
 38: 'L',
 39: 'M',
 40: 'N',
 41: 'O',
 42: 'P',
 43: 'Q',
 44: 'R',
 45: 'S',
 46: 'T',
 47: 'U',
 48: 'V',
 49: 'W',
 50: 'X',
 51: 'Y',
 52: 'Z',
 53: "'",
 54: ' '}

In [287]:
decoded_message = ""
for i in list_of_binary_strings_7bit:
    decoded_message = decoded_message + decoding_map_from_csv[i]

KeyError: '0011110'

In [ ]:
decoded_message

"Don't talk about the club"